In [1]:
import pandas as pd
import numpy as np
ApparelData = pd.read_csv('apparel-trainval.csv',sep=',',index_col = False)
from tqdm import tqdm 

In [2]:
#from tqdm import tqdm

# DIMENSIONALITY REDUCTION

In [3]:
Xmatrix = ApparelData.values
LabelVector = Xmatrix[:,0]
Xmatrix = Xmatrix[:,1:]
meanVector = np.mean(Xmatrix,axis = 0)
CenteredXmatrix = Xmatrix - meanVector
SdVector = np.std(CenteredXmatrix.astype(float),axis = 0)
CenteredXmatrix /= SdVector
#print('calculating the covariance matrix')
covOfCenteredXmatrix = np.array([(CenteredXmatrix[0,:].astype(float))]).T@ np.array([(CenteredXmatrix[0,:].astype(float))])
for i in tqdm(range(1,len(CenteredXmatrix))):
    covOfCenteredXmatrix +=   np.array([(CenteredXmatrix[i,:].astype(float))]).T@ np.array([(CenteredXmatrix[i,:].astype(float))])
covOfCenteredXmatrix /= len(CenteredXmatrix)
eigenValues, eigenVectors = np.linalg.eig(covOfCenteredXmatrix)
idx = eigenValues.argsort()[::-1]   
eigenValues = eigenValues[idx]
eigenVectors = eigenVectors[:,idx]
neededEigenVectors = []
for idx, val in enumerate(eigenValues):
    if (sum(eigenValues[0:idx])/sum(eigenValues))>0.8:
        neededEigenVectors = eigenVectors[:,0:idx]
        break
reconstructedXmatrix = CenteredXmatrix.astype(float)@neededEigenVectors

100%|███████████████████████████████████| 59999/59999 [08:35<00:00, 116.45it/s]


In [4]:
reconstructedXmatrix.shape

(60000, 137)

In [5]:
reconstructedXmatrix = np.insert(reconstructedXmatrix.astype(str), 0, values=LabelVector, axis=1) 

In [6]:
reconstructedXmatrix.shape

(60000, 138)

In [7]:
newReconstructedData = pd.DataFrame(data=reconstructedXmatrix, columns=list(ApparelData.columns)[:138]) 

In [10]:
type(newReconstructedData)

pandas.core.frame.DataFrame

In [22]:
#newReconstructedData.to_csv('ReconstructedApparelData.csv',sep=',',index = False)

In [4]:
newReconstructedData = pd.read_csv('ReconstructedApparelData.csv',sep=',',index_col = False)

In [5]:
newReconstructedData.shape

(60000, 138)

# SCIKIT LEARN BINARY SVM IMPLEMENTATION

In [14]:
UniqueLabels=np.array([5,6])
DataPerClass = []
index = 0
for i in UniqueLabels:
    tempdf = newReconstructedData.loc[newReconstructedData['label'] == i]
    DataPerClass.append(tempdf.sample(frac = 1))
    print(len(DataPerClass[index]))#+str(" for ")+str(i))
    index = index + 1

6000
6000


In [15]:
train5 = DataPerClass[0].sample(frac=0.8)
train6 = DataPerClass[1].sample(frac=0.8)
val5 = DataPerClass[0].loc[~DataPerClass[0].index.isin(train5.index)]
val6 = DataPerClass[1].loc[~DataPerClass[1].index.isin(train6.index)]
allTrain = [train5,train6]
allVal = [val5,val6]
train = pd.concat(allTrain, ignore_index= True)
val = pd.concat(allVal, ignore_index= True)
train = train.sample(frac=1)
val = val.sample(frac=1)
print(len(train))
print(len(val))

9600
2400


In [71]:
#train.to_csv('train56.csv',sep=',',index = False)
#val.to_csv('val56.csv',sep=',',index = False)

In [72]:
train = pd.read_csv('train56.csv',sep=',',index_col = False)
val = pd.read_csv('val56.csv',sep=',',index_col = False)

In [16]:
from sklearn.svm import SVC
clf = SVC(C = 10, kernel = 'linear')
clf.fit(train.values[:,1:], train.values[:,0]) 

print('w = ',clf.coef_)
print('b = ',clf.intercept_)
print('Indices of support vectors = ', clf.support_)
print('Support vectors = ', clf.support_vectors_)
print('Number of support vectors for each class = ', clf.n_support_)
print('Coefficients of the support vector in the decision function = ', np.abs(clf.dual_coef_))

w =  [[ 3.92648099e-01 -2.31049274e-01 -3.49478393e-01  9.04664922e-02
  -2.21604156e-01 -6.16986589e-02  3.10454677e-02  1.21304658e-01
  -8.91354510e-02 -3.37976691e-03 -6.33529227e-02  7.01526203e-02
  -1.04320133e-02  1.59072382e-01  1.37630963e-01 -1.07215887e-01
  -6.95949154e-03  9.03795940e-03  1.24326944e-01  3.96385558e-02
   5.14104630e-02  8.52390017e-02 -9.99873446e-02  8.83880853e-02
  -3.04991569e-02  4.14782929e-02 -5.83555794e-02 -1.02536665e-01
   7.26377740e-02 -5.86025866e-02  1.16889985e-01 -6.68529625e-02
  -8.45425680e-02 -2.40495912e-04 -9.47200483e-02 -6.94441839e-02
  -5.05978581e-02 -7.10931654e-02 -1.07726518e-01 -6.30912541e-02
   2.58953405e-02  1.17382299e-01 -9.46422294e-02 -5.20400330e-02
   3.43444454e-03  1.22803482e-01  5.21399284e-02  1.04550309e-01
  -4.18048012e-02 -1.65499067e-02  5.39902883e-02 -7.30248698e-02
  -1.08893638e-01 -4.27312146e-02  5.05666455e-02  4.47411897e-02
   6.36727466e-02 -8.60428658e-02 -2.96167424e-02 -1.02534400e-01
   9.

In [17]:
PredSVM = []
for i in range(0,len(val)):
    PredSVM.append(clf.predict([val.values[i,1:]]))

In [18]:
PredSVM1 = np.array(PredSVM).ravel()

In [20]:
conMatrix = pd.crosstab(PredSVM1,val.values[:,0])

In [21]:
conMatrix

col_0,5.0,6.0
row_0,,
5.0,1197,1
6.0,3,1199


# CVXOPT BINARY SVM IMPLEMENTATION

In [33]:
from cvxopt import matrix as cvxopt_matrix
from cvxopt import solvers as cvxopt_solvers

In [29]:
train.values[:,0]

array([5., 5., 6., ..., 5., 6., 6.])

In [77]:
#Initializing values and computing H. Note the 1. to force to float type
C = 10
m,n = train.values[:,1:].shape
y = train.values[:,0].reshape(-1,1) * 1.
print(y.shape)
yValues = []
for p in train.values[:,0]:
    if p==5:
        yValues.append(1.0)
    else:
        yValues.append(-1.0)

X = train.values[:,1:]
y = np.array(yValues).reshape(9600,1)
X_dash = y * X

H = np.dot(X_dash , X_dash.T) * 1.

#Converting into cvxopt format - as previously
P = cvxopt_matrix(H)
q = cvxopt_matrix(-np.ones((m, 1)))
G = cvxopt_matrix(np.vstack((np.eye(m)*-1,np.eye(m))))
h = cvxopt_matrix(np.hstack((np.zeros(m), np.ones(m) * C)))
A = cvxopt_matrix(y.reshape(1, -1))
b = cvxopt_matrix(np.zeros(1))

#Run solver
sol = cvxopt_solvers.qp(P, q, G, h, A, b,solver = 'glpk')
alphas = np.array(sol['x'])

#==================Computing and printing parameters===============================#
w = ((y * alphas).T @ X).reshape(-1,1)
S = (alphas > 1e-4).flatten()
b = y[S] - np.dot(X[S], w)

#Display results
print('Alphas = ',alphas[alphas > 1e-4])
print('w = ', w.flatten())
print('b = ', b[0])

(9600, 1)
     pcost       dcost       gap    pres   dres
 0: -4.6249e+03 -8.7107e+05  3e+06  1e+00  5e-11
 1: -2.7081e+03 -3.6738e+05  7e+05  2e-01  4e-11
 2: -9.1305e+02 -9.5808e+04  2e+05  5e-02  3e-11
 3: -5.8545e+02 -6.8172e+04  1e+05  3e-02  2e-11


KeyboardInterrupt: 

In [35]:
#Initializing values and computing H. Note the 1. to force to float type
C = 10
m,n = train.values[:,1:].shape
y = train.values[:,0].reshape(-1,1) * 1.
print(y.shape)
yValues = []
for p in train.values[:,0]:
    if p==5:
        yValues.append(1.0)
    else:
        yValues.append(-1.0)

X = train.values[:,1:]
y = np.array(yValues).reshape(9600,1)
X_dash = y * X

H = np.dot(X_dash , X_dash.T) * 1.

#Converting into cvxopt format - as previously
P = cvxopt_matrix(H)
q = cvxopt_matrix(-np.ones((m, 1)))
G = cvxopt_matrix(np.vstack((np.eye(m)*-1,np.eye(m))))
h = cvxopt_matrix(np.hstack((np.zeros(m), np.ones(m) * C)))
A = cvxopt_matrix(y.reshape(1, -1))
b = cvxopt_matrix(np.zeros(1))

#Run solver
sol = cvxopt_solvers.qp(P, q, G, h, A, b)
alphas = np.array(sol['x'])

#==================Computing and printing parameters===============================#
w = ((y * alphas).T @ X).reshape(-1,1)
S = (alphas > 1e-4).flatten()
b = y[S] - np.dot(X[S], w)

#Display results
print('Alphas = ',alphas[alphas > 1e-4])
print('w = ', w.flatten())
print('b = ', b[0])

(9600, 1)
     pcost       dcost       gap    pres   dres
 0: -4.6249e+03 -8.7107e+05  3e+06  1e+00  5e-11
 1: -2.7081e+03 -3.6738e+05  7e+05  2e-01  4e-11
 2: -9.1305e+02 -9.5808e+04  2e+05  5e-02  4e-11
 3: -5.8545e+02 -6.8172e+04  1e+05  3e-02  3e-11
 4: -3.9071e+02 -5.1264e+04  9e+04  2e-02  2e-11
 5: -1.4725e+02 -3.5427e+04  6e+04  1e-02  9e-12
 6: -1.6587e-01 -2.4059e+04  4e+04  5e-03  5e-12
 7:  6.5554e+01 -1.2473e+04  2e+04  1e-03  2e-12
 8:  5.5582e+01 -3.7491e+03  5e+03  4e-04  1e-12
 9:  4.2820e+01 -1.2524e+03  2e+03  1e-04  4e-13
10:  4.4544e+01 -8.5911e+02  1e+03  6e-05  3e-13
11:  3.9128e+01 -5.2759e+02  7e+02  3e-05  3e-13
12:  2.9402e+01 -2.0814e+02  3e+02  8e-06  3e-13
13:  6.2990e+00 -1.1670e+01  2e+01  2e-07  3e-13
14:  2.7887e+00 -7.0837e+00  1e+01  7e-08  2e-13
15:  1.9618e+00 -6.5631e+00  9e+00  4e-08  1e-13
16:  8.0789e-01 -3.7074e+00  5e+00  2e-08  9e-14
17:  4.7455e-01 -2.9463e+00  3e+00  1e-08  8e-14
18:  2.3618e-01 -2.6014e+00  3e+00  5e-09  7e-14
19: -4.8200

In [53]:
parametersSVM = {}
parametersSVM['alpha'] = alphas[alphas > 1e-4]
parametersSVM['S'] = S
parametersSVM['w'] =  w.flatten()
parametersSVM['b'] = b[0]

import pickle
pickle_out = open("parametersSVM.pickle","wb")
pickle.dump(parametersSVM, pickle_out)
pickle_out.close()

In [55]:
pickle_in = open("parametersSVM.pickle","rb")
parametersSVM1 = pickle.load(pickle_in)

In [56]:
parametersSVM1

{'alpha': array([0.02418618, 0.01700932, 0.00288376, 0.02362461, 0.04242752,
        0.03339694, 0.01380999, 0.02893108, 0.00258885, 0.0089081 ,
        0.02888506, 0.02396257, 0.02178576, 0.01324422, 0.0010124 ,
        0.03301545, 0.01550188, 0.00147725, 0.0029956 , 0.00041264,
        0.00187282, 0.01905717, 0.10126236, 0.09933146, 0.02200155,
        0.02162545, 0.00572389, 0.02558024, 0.00095737, 0.0067859 ,
        0.05952875, 0.02297306, 0.00437663, 0.01037387, 0.00806183,
        0.04044856, 0.0027189 , 0.02301621, 0.01991731, 0.0063148 ,
        0.0047928 , 0.01353949, 0.00954621, 0.01454587, 0.02336076,
        0.01124266, 0.00993328, 0.00510545, 0.08890884, 0.00353605,
        0.02111288, 0.00430706, 0.00469902, 0.01489166, 0.00327372,
        0.00455544, 0.02659261, 0.00521606, 0.03611366, 0.02471124,
        0.02696556, 0.01041876, 0.0033108 , 0.00043027, 0.00686464,
        0.00503842, 0.00834078, 0.00238717, 0.00210083]),
 'S': array([False, False, False, ..., False, Fal

In [57]:
parametersSVM1['w'].shape

(137,)

In [58]:
val.values[i,1:].shape

(137,)

In [59]:
parametersSVM1['b'].shape

(1,)

In [60]:
(parametersSVM1['w'].T@val.values[i,1:])+parametersSVM1['b']

array([3.33319565])

In [61]:
PredSVM2 = []
for i in range(0,len(val)):
    temp1 = (parametersSVM1['w'].T@val.values[i,1:])+parametersSVM1['b']
    if temp1 > 0:
        PredSVM2.append(5)
    else:
        PredSVM2.append(6)

In [66]:
len(np.array(PredSVM2))

2400

In [69]:
len(val.values[:,0])

2400

In [70]:
conMatrix = pd.crosstab(np.array(PredSVM2),val.values[:,0])
conMatrix

col_0,5.0,6.0
row_0,,
5,1197,1
6,3,1199



# MULTI CLASS SVM 

# DUAL FORMULATION

In [75]:
import pandas as pd
import numpy as np
ApparelData = pd.read_csv('apparel-trainval.csv',sep=',',index_col = False)
from tqdm import tqdm 

In [76]:
newReconstructedData = pd.read_csv('ReconstructedApparelData.csv',sep=',',index_col = False)

In [77]:
newReconstructedData.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel128,pixel129,pixel130,pixel131,pixel132,pixel133,pixel134,pixel135,pixel136,pixel137
0,2,10.664973,14.993363,0.689468,10.980911,-4.788254,0.559623,2.141115,2.582798,-4.525370,...,1.231144,-1.230329,-1.386112,1.329886,-0.078191,-1.848467,-0.002599,-0.002010,0.730315,1.324559
1,9,-11.989748,11.812770,5.801049,3.418629,4.630650,2.061772,-3.109565,3.308504,-4.461980,...,-1.254702,-0.316795,-0.121830,0.771904,-1.098454,0.762716,-0.296107,0.191831,-0.115508,0.746862
2,6,20.517671,1.579784,-6.770122,2.884371,5.379185,2.618096,-0.246181,2.936941,3.415353,...,-0.058652,-0.250039,0.363645,-0.580938,0.246680,0.411300,-0.615495,-0.203482,0.286656,-0.199483
3,0,9.634535,-6.790917,0.907860,-4.577228,-8.377438,-0.094376,-8.672806,0.604500,-2.566944,...,-0.733568,-0.737954,0.839011,-1.039877,0.135285,0.579962,-1.163823,-0.044843,-1.631467,-0.366747
4,3,11.493442,-11.655488,7.208334,4.772964,0.594114,0.553685,0.388620,0.384219,2.063610,...,-0.054381,0.290273,-0.522406,-0.841872,0.088202,-0.049652,-0.125279,0.428250,-0.272701,-0.745442


In [451]:
#UniqueLabels=np.array([0,1,2,3,4,5,6,7,8,9])
UniqueLabels=np.array([0,1,2])
DataPerClass = []
index = 0
for i in UniqueLabels:
    tempdf = newReconstructedData.loc[newReconstructedData['label'] == i]
    DataPerClass.append(tempdf.sample(frac = 0.1))
    print(len(DataPerClass[index]))#+str(" for ")+str(i))
    index = index + 1
num_classes = len(UniqueLabels)

600
600
600


In [452]:
train = {}
val = {}
for i in range(0,num_classes):
    train[i] =  DataPerClass[i].sample(frac=0.8)
    val[i]   =  DataPerClass[i].loc[~DataPerClass[i].index.isin(train[i].index)]
allTrain = []
allVal = [] 
for i in range(0,num_classes):
    allTrain.append(train[i])
    allVal.append(val[i])
train1 = pd.concat(allTrain, ignore_index= True)
val1 = pd.concat(allVal, ignore_index= True)
#train1 = train1.sample(frac=1)
#val1 = val1.sample(frac=1)
print(len(train1))
print(len(val1))
trainLabelVector = train1.values[:,0]
valLabelVector = val1.values[:,0]

1440
360


In [453]:
C = 1
#m,n = train.values[:,1:].shape
X = train1.values[:,1:]
X.shape

(1440, 137)

for index,value in enumerate(trainLabelVector):
    if(value==5):
        trainLabelVector[index] = 0
    elif(value==6):
        trainLabelVector[index] = 1
    elif(value==9):
        trainLabelVector[index] = 2

In [454]:
print('hello',flush=True)

hello


In [455]:
np.random.rand(1).astype(float)[0]

0.887865609581771

In [476]:
fullXmatrix =[]
for i in tqdm(range(0,X.shape[0])):
    for p in range(0,num_classes):
        if(trainLabelVector[i]==p):
            #print('hey i am here')
            #pass
            tempArray = []
            for j in range(0,num_classes):
                for k in range(0,X.shape[1]):
                    tempArray.append(np.random.rand(1).astype(float)[0])
                tempArray.append(1.0)
        else:     
            #print('hey i am here')
            tempArray = []
            for j in range(0,num_classes):
                if(j==trainLabelVector[i]):
                    for k in range(0,X.shape[1]):
                        tempArray.append(X[i][k])
                    tempArray.append(1.0)
                elif(j==p):
                    for k in range(0,X.shape[1]):
                        tempArray.append(-X[i][k])
                    tempArray.append(-1.0)                    
                else:
                    for k in range(0,X.shape[1]):
                        tempArray.append(0.0)
                    tempArray.append(0.0)
        fullXmatrix.append(np.array(tempArray))
fullXmatrix = np.array(fullXmatrix)



  0%|                                                 | 0/1440 [00:00<?, ?it/s]

 11%|███▉                                 | 154/1440 [00:00<00:00, 1525.63it/s]

 21%|███████▌                             | 296/1440 [00:00<00:00, 1487.91it/s]

 30%|███████████▏                         | 436/1440 [00:00<00:00, 1456.01it/s]

 41%|███████████████                      | 586/1440 [00:00<00:00, 1464.90it/s]

 52%|███████████████████                  | 742/1440 [00:00<00:00, 1488.23it/s]

 62%|███████████████████████▏             | 900/1440 [00:00<00:00, 1510.52it/s]

 73%|██████████████████████████▍         | 1057/1440 [00:00<00:00, 1523.70it/s]

 84%|██████████████████████████████▎     | 1212/1440 [00:00<00:00, 1527.22it/s]

 95%|██████████████████████████████████  | 1363/1440 [00:00<00:00, 1517.63it/s]

100%|████████████████████████████████████| 1440/1440 [00:00<00:00, 1488.45it/s]

fullXmatrix =[]
for i in tqdm(range(0,X.shape[0])):
    for p in range(0,num_classes):
        if(trainLabelVector[i]==p):
            pass
        else:            
            tempArray = []
            for j in range(0,num_classes):
                if(j==trainLabelVector[i]):
                    for k in range(0,X.shape[1]):
                        tempArray.append(X[i][k])
                    tempArray.append(1.0)
                elif(j==p):
                    for k in range(0,X.shape[1]):
                        tempArray.append(-X[i][k])
                    tempArray.append(-1.0)                    
                else:
                    for k in range(0,X.shape[1]):
                        tempArray.append(0.0)
                    tempArray.append(0.0)
            fullXmatrix.append(tempArray)
fullXmatrix = np.array(fullXmatrix)

fullXmatrix =[]
for i in tqdm(range(0,X.shape[0])):
    tempArray = []
    for j in range(0,num_classes):
        if(j==trainLabelVector[i]):
            #print(j)
            #print(LabelVector[i])
            for k in range(0,X.shape[1]):
                tempArray.append(X[i][k])
            tempArray.append(1)
        else:
            for k in range(0,X.shape[1]):
                tempArray.append(-X[i][k]/(num_classes - 1))
            tempArray.append(-1/(num_classes - 1))
    fullXmatrix.append(tempArray)
fullXmatrix = np.array(fullXmatrix)

In [477]:
fullXmatrix.shape

(4320, 414)

In [478]:
H = fullXmatrix@fullXmatrix.T
m = H.shape[0]
m

4320

In [479]:
H.shape

(4320, 4320)

In [480]:
m

4320

In [481]:
from cvxopt import matrix as cvxopt_matrix
from cvxopt import solvers as cvxopt_solvers

(np.vstack((np.eye(3)*-1,np.eye(3))))
np.eye(2)
(np.hstack((np.zeros(3), np.ones(3) * C)))
#cvxopt_solvers.options['maxiters'] = 3
#cvxopt_solvers.options['abstol'] = 1e-3
#cvxopt_solvers.options['reltol'] = 1e-3
#cvxopt_solvers.options['feastol'] = 1e-3
#cvxopt_solvers.options['show_progress'] = True
#opts = {'maxiters' : 10}

In [482]:
G = cvxopt_matrix(np.vstack((np.eye(m)*-1,np.eye(m))))
h = cvxopt_matrix(np.hstack((np.zeros(m), np.ones(m) * C)))

Amatrix = np.zeros((m,m))
for i in tqdm(range(0,len(train))):
    for x in range(0,Amatrix.shape[0]):
        if((x<((i+1)*num_classes)) and (x>=((i)*num_classes))):
            for y in range(0,Amatrix.shape[1]):
                if((y<((trainLabelVector[i]+1)*num_classes)) and (y>=((trainLabelVector[i])*num_classes))):
                    if(x==y):
                        Amatrix[x,y]= 1.0        
                elif(y>((trainLabelVector[i]+1)*num_classes)):
                    break
        elif(x>((i+1)*num_classes)):
            break
#for (x,y), value in np.ndenumerate(Amatrix):

In [483]:
Amatrix =[]
for i in tqdm(range(0,len(train1))):
    #print(i)
    #print(m)
    #print(int((i*num_classes)+trainLabelVector[i]))
    temp2 = [0.0]*m
    temp2[int((i*num_classes)+trainLabelVector[i])] = 1.0
    Amatrix.append(temp2)
Amatrix = np.array(Amatrix)



  0%|                                                 | 0/1440 [00:00<?, ?it/s]

100%|███████████████████████████████████| 1440/1440 [00:00<00:00, 13853.99it/s]

In [375]:
Amatrix

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [484]:
A = cvxopt_matrix(Amatrix)
b = cvxopt_matrix(np.zeros(Amatrix.shape[0]))
q = cvxopt_matrix(-1*np.ones((m, 1)))
P = cvxopt_matrix(H)

In [485]:
opts = {'maxiters' : 30,'solver':'mosek'}
sol = cvxopt_solvers.qp(P, q, G, h, options = opts)#, A, b,solver = 'glpk')
alphas2 = np.array(sol['x'])

     pcost       dcost       gap    pres   dres
 0: -1.7188e+03 -1.1115e+04  6e+04  3e+00  6e-12
 1: -1.2243e+03 -7.7202e+03  1e+04  5e-01  5e-12
 2: -1.1767e+03 -4.3378e+03  5e+03  2e-01  3e-12
 3: -1.2671e+03 -2.7732e+03  2e+03  6e-02  2e-12
 4: -1.3310e+03 -2.2283e+03  1e+03  3e-02  1e-12
 5: -1.3620e+03 -1.9730e+03  9e+02  2e-02  1e-12
 6: -1.3667e+03 -1.8476e+03  7e+02  1e-02  7e-13
 7: -1.4013e+03 -1.6238e+03  3e+02  4e-03  5e-13
 8: -1.4326e+03 -1.4828e+03  6e+01  7e-04  3e-13
 9: -1.4365e+03 -1.4655e+03  3e+01  3e-04  3e-13
10: -1.4402e+03 -1.4525e+03  1e+01  1e-04  2e-13
11: -1.4419e+03 -1.4469e+03  5e+00  1e-05  3e-13
12: -1.4426e+03 -1.4455e+03  3e+00  5e-06  3e-13
13: -1.4431e+03 -1.4445e+03  1e+00  2e-16  3e-13
14: -1.4435e+03 -1.4439e+03  4e-01  2e-16  3e-13
15: -1.4437e+03 -1.4437e+03  4e-02  2e-16  3e-13
16: -1.4437e+03 -1.4437e+03  1e-03  2e-16  3e-13
Optimal solution found.


import pickle
pickle_out = open("alphaMulticlassSVM.pickle","wb")
pickle.dump(alphas, pickle_out)
pickle_out.close()

pickle_in = open("alphaMulticlassSVM.pickle","rb")
alphas1 = pickle.load(pickle_in)

for i in range(0,len(train)):
    for j in range(0,num_classes):
        if(j==trainLabelVector[i]):        
            alphas3[(num_classes*i)+j] = 0

In [486]:
fullXmatrix.shape

(4320, 414)

In [487]:
w = fullXmatrix.T@alphas2

In [488]:
len(alphas2[alphas2>10**-6])

1591

In [489]:
#w = (fullXmatrix*(alphas2))#.reshape(-1,1)

In [490]:
#w = w.sum(axis = 0).reshape(w.shape[1],1)

In [491]:
w.shape

(414, 1)

In [492]:
print('hello')

hello


In [493]:
wForEachLabel = {}
for i in range(0,num_classes):
    #wForEachLabel[i] = w[i:i+(X.shape[1]),:]    
    wForEachLabel[i] = w[i:i+(X.shape[1])+1,:]    
predictedLabelVector = []
for i in tqdm(range(0,len(val1))):
    scoresForEachLabel = []
    for j in range(0,num_classes):
        #score = (wForEachLabel[j].T@val1.values[i,1:].reshape(-1,1))
        score = (wForEachLabel[j].T@np.append(val1.values[i,1:],1.0).reshape(-1,1))
        scoresForEachLabel.append(score)
    maxScoreLabel = np.argmax(scoresForEachLabel)
    print(scoresForEachLabel)
    print(maxScoreLabel)
    predictedLabelVector.append(maxScoreLabel)



  0%|                                                  | 0/360 [00:00<?, ?it/s]

[array([[3.22980373]]), array([[-1.76865528]]), array([[3.22073021]])]
0
[array([[5.4309691]]), array([[1.52988537]]), array([[8.16981]])]
2
[array([[4.33243798]]), array([[-2.86957738]]), array([[4.64978436]])]
2
[array([[5.45352857]]), array([[-1.7793523]]), array([[7.00541007]])]
2
[array([[1.74669161]]), array([[-2.1852137]]), array([[4.14456975]])]
2
[array([[4.67197022]]), array([[-3.3366582]]), array([[10.4404482]])]
2
[array([[4.39076006]]), array([[-4.97482733]]), array([[7.55416244]])]
2
[array([[7.70261272]]), array([[-1.44158253]]), array([[9.20691609]])]
2
[array([[6.11210777]]), array([[-2.93971656]]), array([[7.64328283]])]
2
[array([[1.66383878]]), array([[1.4535292]]), array([[1.08893992]])]
0
[array([[2.83041529]]), array([[-2.44415345]]), array([[5.83109475]])]
2
[array([[4.72791711]]), array([[-3.47439076]]), array([[5.78525011]])]
2
[array([[3.19372246]]), array([[-7.05935053]]), array([[5.56404368]])]
2
[array([[5.16570658]]), array([[-2.30345148]]), array([[9.338



  6%|██▏                                     | 20/360 [00:00<00:01, 190.58it/s]

[array([[3.41698195]]), array([[-11.05052181]]), array([[7.38483247]])]
2
[array([[2.47495025]]), array([[-6.2139649]]), array([[7.32847737]])]
2
[array([[1.75097725]]), array([[2.60844807]]), array([[3.87788583]])]
2
[array([[5.9519653]]), array([[-7.32054884]]), array([[11.77479438]])]
2
[array([[4.71003135]]), array([[0.04594543]]), array([[8.16585604]])]
2
[array([[1.37116774]]), array([[-0.64333888]]), array([[2.90320447]])]
2
[array([[6.20684183]]), array([[-2.57896599]]), array([[7.16854586]])]
2
[array([[3.34694336]]), array([[-5.11132389]]), array([[6.0230286]])]
2
[array([[3.15476271]]), array([[-10.64706246]]), array([[6.20155222]])]
2
[array([[3.93854023]]), array([[-9.92235238]]), array([[1.90699894]])]
0
[array([[-1.64267095]]), array([[2.24396897]]), array([[-0.76487705]])]
1
[array([[2.79923166]]), array([[-2.02871958]]), array([[4.12378725]])]
2
[array([[6.49210693]]), array([[-1.16338605]]), array([[7.34672785]])]
2
[array([[3.00209487]]), array([[0.1983938]]), array(



 11%|████▎                                   | 39/360 [00:00<00:01, 187.62it/s]

...



 92%|████████████████████████████████████   | 333/360 [00:02<00:00, 129.79it/s]

[array([[-0.82300019]]), array([[3.38045866]]), array([[1.68329212]])]
1
[array([[-3.82816356]]), array([[7.48327212]]), array([[0.36420397]])]
1
[array([[-2.78070686]]), array([[5.96484489]]), array([[4.5312799]])]
1
[array([[-1.85576856]]), array([[4.75719364]]), array([[9.64687023]])]
2
[array([[-4.99883489]]), array([[1.15320181]]), array([[1.25384507]])]
2
[array([[-0.40330757]]), array([[4.23248368]]), array([[1.08029584]])]
1
[array([[-2.37284]]), array([[2.83620313]]), array([[-1.67948815]])]
1
[array([[-3.12982977]]), array([[10.26818583]]), array([[6.67124378]])]
1
[array([[-0.80445766]]), array([[5.09368343]]), array([[3.6709292]])]
1
[array([[-0.05689828]]), array([[-0.34292215]]), array([[0.31382184]])]
2
[array([[1.24509895]]), array([[-2.20145484]]), array([[1.72239914]])]
2
[array([[1.30214577]]), array([[-2.23909385]]), array([[3.02651967]])]
2
[array([[-2.80109788]]), array([[6.82219527]]), array([[5.81508125]])]
1
[array([[-1.41531438]]), array([[2.52786989]]), array



 97%|██████████████████████████████████████▋ | 348/360 [00:02<00:00, 95.14it/s]

[array([[-4.17992467]]), array([[-0.20765542]]), array([[2.54637685]])]
2
[array([[-3.99694208]]), array([[1.62201719]]), array([[2.97584678]])]
2
[array([[-3.62998412]]), array([[7.07687696]]), array([[1.78057099]])]
1
[array([[-0.23254679]]), array([[-0.35226463]]), array([[2.73142324]])]
2
[array([[-1.02026923]]), array([[-0.65095997]]), array([[0.26786881]])]
2
[array([[-0.57244533]]), array([[2.47164112]]), array([[4.86749078]])]
2
[array([[-1.99864316]]), array([[9.02644782]]), array([[0.7788419]])]
1
[array([[-0.62873995]]), array([[0.77363168]]), array([[1.21995767]])]
2
[array([[-1.04280825]]), array([[3.33404022]]), array([[6.35462641]])]
2
[array([[-1.15581945]]), array([[6.19830653]]), array([[0.79902319]])]
1
[array([[-2.95743381]]), array([[2.32102548]]), array([[2.10035145]])]
1
[array([[-3.67952444]]), array([[1.08909446]]), array([[-1.52134493]])]
1




100%|███████████████████████████████████████| 360/360 [00:02<00:00, 120.31it/s]

In [494]:
for i in range(0,num_classes):
    print(wForEachLabel[i].shape)

(138, 1)
(138, 1)
(138, 1)


for index,value in enumerate(ValLabelVector):
    if(value==5):
        ValLabelVector[index] = 0.0
    elif(value==6):
        ValLabelVector[index] = 1.0
    elif(value==9):
        ValLabelVector[index] = 2.0

In [495]:
valLabelVector

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.

In [496]:
np.array(predictedLabelVector)

array([0, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 0, 1, 2, 1, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 0, 1, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 0, 2, 2, 0, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 0,
       2, 2, 2, 2, 2, 2, 2, 0, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 1, 1,
       1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 2, 1, 1,
       1, 1, 1, 0, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 2, 1, 1, 2, 2, 1, 1,
       1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2,

In [497]:
conMatrix = pd.crosstab(np.array(predictedLabelVector).ravel(),valLabelVector.reshape(-1,1).ravel())

In [498]:
conMatrix

col_0,0.0,1.0,2.0
row_0,,,
0,17,2,1
1,5,0,87
2,98,118,32


In [54]:
from sklearn import svm

In [55]:
clf = svm.SVC(gamma='scale', decision_function_shape='ovo',kernel='linear')
clf.fit(X, trainLabelVector) 

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovo', degree=3, gamma='scale', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [56]:
X.shape

(480, 137)

In [57]:
lin_clf.decision_function(X).shape

(480, 10)

In [241]:
clf.predict(val.values[0,1:].reshape(1,137))

array([5.])

In [242]:
predictedLabelVector = []
for i in tqdm(range(0,len(val))):
    predictedLabelVector.append(clf.predict(val.values[i,1:].reshape(1,137)))


100%|██████████████████████████████████████| 120/120 [00:00<00:00, 1052.83it/s]

In [243]:
np.array(predictedLabelVector).ravel().shape

(120,)

In [244]:
valLabelVector.reshape(-1,1).ravel().shape

(120,)

In [245]:
conMatrix = pd.crosstab(np.array(predictedLabelVector).ravel(),valLabelVector.reshape(-1,1).ravel())

In [246]:
conMatrix

col_0,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0
row_0,,,,,,,,,,
0.0,11,0,0,0,0,0,1,0,0,0
1.0,1,11,0,0,0,0,0,0,0,0
2.0,1,0,8,0,3,0,0,0,0,0
3.0,3,0,0,9,0,0,0,0,0,0
4.0,0,0,2,1,8,0,1,0,0,0
5.0,0,0,0,0,0,11,0,1,0,0
6.0,2,0,1,0,3,0,6,0,0,0
7.0,0,0,0,0,0,1,0,10,0,1
8.0,0,0,0,0,2,0,0,0,10,0


In [248]:
from sklearn import svm
lin_clf = svm.LinearSVC()
lin_clf.max_iter=10**5
lin_clf.fit(X, trainLabelVector) 

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=100000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [249]:
lin_clf.decision_function(X).shape

(1920, 4)

In [251]:
predictedLabelVector = []
for i in tqdm(range(0,len(val1))):
    predictedLabelVector.append(lin_clf.predict(val1.values[i,1:].reshape(1,137)))

100%|██████████████████████████████████████| 480/480 [00:00<00:00, 5336.43it/s]


In [252]:
conMatrix = pd.crosstab(np.array(predictedLabelVector).ravel(),valLabelVector.reshape(-1,1).ravel())

In [253]:
conMatrix

col_0,0.0,1.0,2.0,3.0
row_0,,,,
0.0,106,0,1,6
1.0,3,116,2,6
2.0,4,0,115,3
3.0,7,4,2,105
